In [1]:
import sys

In [2]:
import findspark
findspark.init()
import pyspark
import random

In [3]:
from elasticsearch import Elasticsearch

In [6]:
es = Elasticsearch()

#### Run the follwing command in the terminal(to authenticate) to get the mail archive before running the next commands
gmvault sync you@gmail.com -d path/to/where/you/want/the/email/archived

In [11]:
!cat /Users/GodSpeed/Documents/mailArchive/db/2010-10/1349228336576455316.meta

{"thread_ids": 1349228336576455316, "msg_id": "1785262635.1286724410050.JavaMail.async@sl-prod-sso-eas5-1", "flags": ["\\Seen"], "gm_id": 1349228336576455316, "internal_date": 1286706411, "x_gmail_received": null, "labels": ["\\Inbox"], "subject": "Welcome to your Nokia account"}

In [16]:
!gzcat /Users/GodSpeed/Documents/mailArchive/db/2010-10/1349228336576455316.eml.gz

### ===================================
We’ll walk all the files in the root of the Gmvault database using os.walk, find all files that end in .meta, load the JSON in those files, tweak the JSON just a bit (more on that in a second), and then shove the JSON into Elasticsearch.

In [48]:
import os, json, timeit
import email
import gzip
from tqdm import tqdm

In [67]:
#can just store the mapping keys instead of the whole document,and when searched, 
#may access the data using the mapping key. This may be done by storing the actual data somewhere else, 
#maybe hadoop or not

def fileToES(folder_path):
    for parent_folder, folders, files in os.walk(folder_path):
        f = FloatProgress(min = 0, max = len(files))
        display(f)
        for filename in files:
            f.value += 1
            if filename.endswith('.meta'):
                with gzip.open(os.path.join(parent_folder, filename.rsplit('.', 1)[0]+'.eml.gz'), 'r') as fp:
                    message = email.message_from_file(fp)
                #change all the content to unicode for convenience of elasticsearch
                meta = {toUnicode(key).lower(): toUnicode(value) 
                        for key, value in message.items()}
                #we'll only yake from and to keys from the mail headers
                #requiredKeys = ['from', 'to']
                #meta = {key: messageItemsKeyValue[key] for key in requiredKeys}
                #print meta
                with open(os.path.join(parent_folder, filename), 'r') as fp:
                    meta.update(json.load(fp))
                    #meta = json.load(fp)
                #add additional attributes to meta[_'source'], 
                #all new additions are made implicitly inside the meta['_source']
                meta['account'] = 'rohit.mlhtra@gmail.com'
                meta['path'] = os.path.join(parent_folder, filename)
                es.index(index = 'mail', doc_type = 'message', id = meta['gm_id'], body=meta)
                #print meta
                #break

In [68]:
from ipywidgets import FloatProgress
from IPython.display import display

folder_path = '/Users/GodSpeed/Documents/mailArchive/'
fileToES(folder_path)

## Searching the metadata

##### Lazy search(non structured query)

In [47]:
es.search('mail', q='playstation')

{u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'1574119758260325826',
    u'_index': u'mail',
    u'_score': 9.661735,
    u'_source': {u'account': u'rohit.mlhtra@gmail.com',
     u'flags': [u'\\Seen'],
     u'gm_id': 1574119758260325826,
     u'internal_date': 1501179584,
     u'labels': [u'\\Inbox'],
     u'msg_id': u'0100015d86594699-8fb526c7-a6dc-4bfd-9c3f-7ea2c9febfd5-000000@email.amazonses.com',
     u'path': u'/Users/GodSpeed/Documents/mailArchive/db/2017-07/1574119758260325826.meta',
     u'subject': u'PlayStation 4 Slim 500GB...',
     u'thread_ids': 1574119758260325826,
     u'x_gmail_received': None},
    u'_type': u'message'},
   {u'_id': u'1582925169382463414',
    u'_index': u'mail',
    u'_score': 9.443269,
    u'_source': {u'account': u'rohit.mlhtra@gmail.com',
     u'arc-authentication-results': u'i=1; mx.google.com;\r\n       dkim=pass header.i=@amazon.com header.s=v52wzeq2nhi2ivaga2mkkcxigoeqeycs header.b=hwo

#### To do: 
1. sync offline mails regularly

#### Parsing emails text

In [17]:
#converting all characters to unicode, as elasticsearch handles only unicode
def toUnicode(char):
    return char.decode('latin-1', errors='replace')

In [18]:
import email
import gzip

In [66]:
!gzcat /Users/GodSpeed/Documents/mailArchive/db/2010-10/1349228336576455316.eml.gz

In [21]:
with gzip.open('/Users/GodSpeed/Documents/mailArchive/db/2010-10/1349228336576455316.eml.gz', 'r') as fp:
    message = email.message_from_file(fp)
print "%r"%(message.items())

[('X-Antivirus', 'avast (VPS 17122204)'), ('X-Antivirus-Status', 'Clean'), ('Delivered-To', 'rohitmlhtra@gmail.com'), ('Received', 'by 10.142.212.7 with SMTP id k7cs25361wfg;\r\n        Sun, 10 Oct 2010 08:26:52 -0700 (PDT)'), ('Received', 'by 10.213.27.136 with SMTP id i8mr2791904ebc.46.1286724411783;\r\n        Sun, 10 Oct 2010 08:26:51 -0700 (PDT)'), ('Return-Path', '<NokiaAccount-no-reply@comms.nokia.com>'), ('Received', 'from saempgwp01.nokia.com (saempgwp01.nokia.com [147.243.1.20])\r\n        by mx.google.com with SMTP id w46si9386858eeh.61.2010.10.10.08.26.50;\r\n        Sun, 10 Oct 2010 08:26:51 -0700 (PDT)'), ('Received-SPF', 'pass (google.com: domain of NokiaAccount-no-reply@comms.nokia.com designates 147.243.1.20 as permitted sender) client-ip=147.243.1.20;'), ('Authentication-Results', 'mx.google.com; spf=pass (google.com: domain of NokiaAccount-no-reply@comms.nokia.com designates 147.243.1.20 as permitted sender) smtp.mail=NokiaAccount-no-reply@comms.nokia.com'), ('Receiv